In [2]:
# this file is to make unified data for all the data
import pandas as pd
import numpy as np
import os

In [3]:
df_news = pd.read_csv('News_U.csv')
df_bond = pd.read_csv('cleaned_bond.csv')
df_call = pd.read_csv('call_rates_3.csv')

In [4]:
# add source column to identify the source of the news
df_news['source'] = df_news['Link'].apply(lambda x: 'infomax' if 'infomax' in x else ('edaily' if 'edaily' in x else 'yh' if 'yna' in x else None))
df_bond['source'] = 'bond'

In [5]:
# change Date column's datatype from object to datetime
df_news['Date'] = pd.to_datetime(df_news['Date'], format='%Y.%m.%d')
df_bond['Date'] = pd.to_datetime(df_bond['Date'], format='%Y.%m.%d')
df_call['Date'] = pd.to_datetime(df_call['Date'], format='%Y.%m.%d')

In [6]:
# drop unnecessary columns before merging
df_bond.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
df_corp = pd.concat([df_news, df_bond], ignore_index=True)
df_corp

Date                                Title  \
0      2014-08-11         <뉴욕환시-주간> 달러, 지정학적 불안에 조정 압박   
1      2014-08-11         8월 금통위 박빙 동결도 가능이주열 결단이 가른다③   
2      2014-08-11        집 사라고 대출규제 풀었지만…집 안사고 생활비로 쓴다   
3      2014-08-11                    유가 혼조지정학적 리스크 제한적   
4      2014-08-11                실적따라 주가차별화유틸리티 등 상승기대   
...           ...                                  ...   
248736 2024-07-18                        추가 금리 하락 룸 bp   
248737 2024-05-07    Kiwoom Fixed Income Weekly Review   
248738 2024-02-13         Daily Bond Morning Brief       
248739 2014-11-03                월  주 글로벌 채권 자산 순유입 지속   
248740 2023-07-18                돌아온 외국인 그래도 조심해야 할 구간   

                                                  Content  \
0        이번 주(11~15일) 뉴욕 외환시장에서 달러화는 지정학적 불안으로 단기적으로 하...   
1       한국은행 7월 금융통화위원회 의사록을 통해 나타난 6명의 금통위원이 본 경기판단은 ...   
2       정부가 주택시장 활성화를 위해 LTVDTI와 같은 대출규제를 풀었지만 정작 은행에선...   
3       국제유가는 지난 8일(현지시간) 제한적인 지정학적 리스크 영향에 혼조 마감했다 금값...   
4       KB투자증권은 국가별 업종별 차별화가 진행되는 가운데 이런 주가 차별화의 원인은 기...   
...                                                   ...   
248736  년 월 일 년 월 일 IGlobal AssetResearchBondView 추가 금...   
248737  ComplianceNotice 당사는 동자료를 기관투자자 또는 제자에게 사전 제공한...   
248738   억원 yyKRM arketKRBond bp YTMDWM  억원 CDD      Y...   
248739  Economic   www deri co krFundflows WeeklyAsset...   
248740  I 채권 전략 년 월 일 돌아온 외국인 그래도 조심해야 할 구간 외국인 국채선물 순...   

                                                     Link   source  
0       https://news.einfomax.co.kr/news/articleView.h...  infomax  
1       https://www.edaily.co.kr/News/Read?newsId=0109...   edaily  
2       https://www.edaily.co.kr/News/Read?newsId=0117...   edaily  
3       https://www.edaily.co.kr/News/Read?newsId=0119...   edaily  
4       https://www.edaily.co.kr/News/Read?newsId=0120...   edaily  
...                                                   ...      ...  
248736  https://stock.pstatic.net/stock-research/deben...     bond  
248737  https://ssl.pstatic.net/imgstock/upload/resear...     bond  
248738  https://ssl.pstatic.net/imgstock/upload/resear...     bond  
248739  http://imgstock.naver.com/upload/research/debe...     bond  
248740  https://ssl.pstatic.net/imgstock/upload/resear...     bond  

[248741 rows x 5 columns]

In [8]:
pk_dict = {'yh': '1', 'edaily': '2', 'infomax': '3', 'bond': '4'}
df_corp['counter'] = df_corp.groupby(['source', 'Date']).cumcount() + 1
df_corp['source_num'] = df_corp['source'].map(pk_dict)
# primary key 생성
df_corp['pk'] =  df_corp['source_num'] + df_corp['Date'].dt.strftime('%Y%m%d') + df_corp['counter'].apply(lambda x: f'{x:03}')
df_corp['pk']

0         320140811001
1         220140811001
2         220140811002
3         220140811003
4         220140811004
              ...     
248736    420240718006
248737    420240507010
248738    420240213009
248739    420141103004
248740    420230718008
Name: pk, Length: 248741, dtype: object

In [9]:
df_corp.columns

Index(['Date', 'Title', 'Content', 'Link', 'source', 'counter', 'source_num',
       'pk'],
      dtype='object')

In [10]:
df_corpus = df_corp[['pk', 'Date', 'Title', 'Content']]
df_corpus

pk       Date                                Title  \
0       320140811001 2014-08-11         <뉴욕환시-주간> 달러, 지정학적 불안에 조정 압박   
1       220140811001 2014-08-11         8월 금통위 박빙 동결도 가능이주열 결단이 가른다③   
2       220140811002 2014-08-11        집 사라고 대출규제 풀었지만…집 안사고 생활비로 쓴다   
3       220140811003 2014-08-11                    유가 혼조지정학적 리스크 제한적   
4       220140811004 2014-08-11                실적따라 주가차별화유틸리티 등 상승기대   
...              ...        ...                                  ...   
248736  420240718006 2024-07-18                        추가 금리 하락 룸 bp   
248737  420240507010 2024-05-07    Kiwoom Fixed Income Weekly Review   
248738  420240213009 2024-02-13         Daily Bond Morning Brief       
248739  420141103004 2014-11-03                월  주 글로벌 채권 자산 순유입 지속   
248740  420230718008 2023-07-18                돌아온 외국인 그래도 조심해야 할 구간   

                                                  Content  
0        이번 주(11~15일) 뉴욕 외환시장에서 달러화는 지정학적 불안으로 단기적으로 하...  
1       한국은행 7월 금융통화위원회 의사록을 통해 나타난 6명의 금통위원이 본 경기판단은 ...  
2       정부가 주택시장 활성화를 위해 LTVDTI와 같은 대출규제를 풀었지만 정작 은행에선...  
3       국제유가는 지난 8일(현지시간) 제한적인 지정학적 리스크 영향에 혼조 마감했다 금값...  
4       KB투자증권은 국가별 업종별 차별화가 진행되는 가운데 이런 주가 차별화의 원인은 기...  
...                                                   ...  
248736  년 월 일 년 월 일 IGlobal AssetResearchBondView 추가 금...  
248737  ComplianceNotice 당사는 동자료를 기관투자자 또는 제자에게 사전 제공한...  
248738   억원 yyKRM arketKRBond bp YTMDWM  억원 CDD      Y...  
248739  Economic   www deri co krFundflows WeeklyAsset...  
248740  I 채권 전략 년 월 일 돌아온 외국인 그래도 조심해야 할 구간 외국인 국채선물 순...  

[248741 rows x 4 columns]

In [11]:
results = []

In [12]:
df_call.columns

Index(['Unnamed: 0', 'Date', 'Call_Rate', 'CD', 'Treasury_Bond_3Y',
       'Treasury_Bond_5Y', 'Corporate_Bond'],
      dtype='object')

In [13]:
def calculate_label(article_date):
    if article_date not in df_call['Date'].values:
        return None
    
    current_rate = df_call.loc[df_call['Date'] == article_date, 'Call_Rate'].values[0]
    next_date = article_date + pd.DateOffset(months=2)
    if next_date not in df_call['Date'].values:
        return None
    
    next_rate = df_call.loc[df_call['Date'] == next_date, 'Call_Rate'].values[0]
    difference = next_rate - current_rate

    if difference > 0.01:
        label = '1'
    elif difference < -0.01:
        label = '0'
    else:
        label = np.nan

    results.append({
        'Date': article_date,
        'Date+2m': next_date,
        'Current_rate': current_rate,
        'Rate+2m': next_rate,
        'Difference': difference,
        'Label': label
    })

    return label

In [14]:
df_corpus['Label'] = df_corpus['Date'].apply(calculate_label)
df_corpus

/var/folders/gd/0dt483qd2cg3cqgcsrx3pnkr0000gp/T/ipykernel_63613/2220490043.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corpus['Label'] = df_corpus['Date'].apply(calculate_label)


pk       Date                                Title  \
0       320140811001 2014-08-11         <뉴욕환시-주간> 달러, 지정학적 불안에 조정 압박   
1       220140811001 2014-08-11         8월 금통위 박빙 동결도 가능이주열 결단이 가른다③   
2       220140811002 2014-08-11        집 사라고 대출규제 풀었지만…집 안사고 생활비로 쓴다   
3       220140811003 2014-08-11                    유가 혼조지정학적 리스크 제한적   
4       220140811004 2014-08-11                실적따라 주가차별화유틸리티 등 상승기대   
...              ...        ...                                  ...   
248736  420240718006 2024-07-18                        추가 금리 하락 룸 bp   
248737  420240507010 2024-05-07    Kiwoom Fixed Income Weekly Review   
248738  420240213009 2024-02-13         Daily Bond Morning Brief       
248739  420141103004 2014-11-03                월  주 글로벌 채권 자산 순유입 지속   
248740  420230718008 2023-07-18                돌아온 외국인 그래도 조심해야 할 구간   

                                                  Content Label  
0        이번 주(11~15일) 뉴욕 외환시장에서 달러화는 지정학적 불안으로 단기적으로 하...  None  
1       한국은행 7월 금융통화위원회 의사록을 통해 나타난 6명의 금통위원이 본 경기판단은 ...  None  
2       정부가 주택시장 활성화를 위해 LTVDTI와 같은 대출규제를 풀었지만 정작 은행에선...  None  
3       국제유가는 지난 8일(현지시간) 제한적인 지정학적 리스크 영향에 혼조 마감했다 금값...  None  
4       KB투자증권은 국가별 업종별 차별화가 진행되는 가운데 이런 주가 차별화의 원인은 기...  None  
...                                                   ...   ...  
248736  년 월 일 년 월 일 IGlobal AssetResearchBondView 추가 금...  None  
248737  ComplianceNotice 당사는 동자료를 기관투자자 또는 제자에게 사전 제공한...  None  
248738   억원 yyKRM arketKRBond bp YTMDWM  억원 CDD      Y...  None  
248739  Economic   www deri co krFundflows WeeklyAsset...  None  
248740  I 채권 전략 년 월 일 돌아온 외국인 그래도 조심해야 할 구간 외국인 국채선물 순...     1  

[248741 rows x 5 columns]

In [15]:
df_corpus.dropna(inplace=True)
df_corpus['Label'].value_counts()

/var/folders/gd/0dt483qd2cg3cqgcsrx3pnkr0000gp/T/ipykernel_63613/164719137.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_corpus.dropna(inplace=True)


Label
1    64815
0    51175
Name: count, dtype: int64

In [16]:
# Get the current working directory
current_dir = os.getcwd()

# Change the directory to the current working directory
df_corpus.to_csv(os.path.join(current_dir, 'corpus_data.csv'), index=False)